### Prior to feature extraction, the Sentinel-2 spectral bands must be downloaded, and the ground truth dataset—including geometries, class labels, and unique identifiers—must be compiled.

In [17]:
import geopandas as gpd
import rasterio
import os
from glob import glob
from natsort import natsorted
from rasterio.mask import mask
import numpy as np
from tqdm import tqdm
import pandas as pd
import ast

In [ ]:
gpkg_path = "input_data.gpkg"
gdf = gpd.read_file(gpkg_path)
gdf.columns

Index(['PLOT_NO', 'label', 'SURVEY_DATE', 'PLANTING_DATE', 'CROPTYPE_CODE',
       '2023', 'farm_id', 'geometry'],
      dtype='object')

In [ ]:
raster_path = natsorted(glob("images/B2/*.tif"))[:-1]
raster_path

In [6]:
date_list = []
for file in raster_path:
    date_list.append(os.path.basename(file)[:8])
date_list

['20250101',
 '20250115',
 '20250201',
 '20250215',
 '20250301',
 '20250315',
 '20250401',
 '20250415',
 '20250501',
 '20250515',
 '20250601',
 '20250615',
 '20250701',
 '20250715',
 '20250801',
 '20250815',
 '20250901',
 '20250915',
 '20251001',
 '20251015',
 '20251101',
 '20251115',
 '20251201']

In [7]:
grand_list = []
for idx, row in tqdm(gdf.iterrows(), total=len(gdf)):
    geom = row.geometry
    label = row.get('label')
    PLOT_NO = row.get('PLOT_NO')
    df_list = []
    time_stack_l = []
    for d in date_list:
        band_stack_l = []
        for band in ["B2", "B3", "B4", "B8"]:
            file = f"images/{band}/{d}_{band}.tif"
            with rasterio.open(file) as src:
                masked_image, _ = mask(src, [geom], crop=True)
                band_stack_l.append(masked_image)
        band_stack = np.concatenate(band_stack_l, axis=0)
        time_stack_l.append(band_stack)
    time_stack = np.stack(time_stack_l, axis=0)
    T, F, H, W = time_stack.shape
    # Conver to time, feature and pixel
    reshaped_time_stack = time_stack.reshape(T, F, H*W)
    # Remove nan values from per temporal pixel
    for i in range(reshaped_time_stack.shape[-1]):
        if np.isnan(reshaped_time_stack[:,:, i]).any():
            continue
        df_list.append(
        [reshaped_time_stack[:,:,i], label]
    )
    df = pd.DataFrame(df_list)
    grand_list.append(df)
grand_df = pd.concat(grand_list)
grand_df.to_pickle("grand_df.pkl")

100%|██████████| 857/857 [09:59<00:00,  1.43it/s]
